In [1]:
import polars as pl
import datetime
import os

from utils.functions import tramit_file_reader, dates_range, simplify_euro_emissions
from utils.dictionaries import types_parque_post, types_tramit_post
from utils.dictionaries import common_cols_tramites, common_cols_mapping, common_cols_parque_exacto

In [ ]:
clearn_park_path = os.path.join('..','Data','DGT','Parque_Exacto','clean_park.csv')
bajas_dir_path = os.path.join('..','Data','DGT','bajas')

common_cols_tramites = ['FEC_MATRICULA','FEC_PRIM_MATRICULACION','COD_PROPULSION_ITV']
common_cols_mapping = {"FEC_MATRICULA": "FECHA_MATR", "FEC_PRIM_MATRICULACION": "FECHA_PRIM_MATR", "COD_PROPULSION_ITV": "PROPULSION"}
necessary_cols = ['FECHA_PRIM_MATR','FECHA_MATR','PROPULSION']

park = pl.scan_csv(clearn_park_path,separator='|', schema=types_parque_post).select(necessary_cols)
park = park.collect()
park = simplify_euro_emissions(park).drop('PROPULSION')

bajas_files = [
    f for f in os.listdir(bajas_dir_path)
    if os.path.isfile(os.path.join(bajas_dir_path, f)) and f.lower().endswith(".txt")
]

first_file_path = os.path.join(bajas_dir_path,bajas_files[0])
file_bajas = tramit_file_reader(first_file_path).select(common_cols_tramites).rename(common_cols_mapping)
file_bajas = simplify_euro_emissions(file_bajas).drop('PROPULSION')

for i in range(1,len(bajas_files)):
    file_path = os.path.join(bajas_dir_path,bajas_files[i])
    new_file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
    new_file = simplify_euro_emissions(new_file).drop('PROPULSION')
    file_bajas = pl.concat([file_bajas,new_file])

file = pl.concat([park,file_bajas])
file_path = os.path.join('..','Data','DGT','Parque_Exacto','park_w_deregisters.csv')
file.write_csv(file_path,separator='|')